# Fake news classifier using LSTM

In [59]:
import pandas as pd

In [60]:
## dataset https://www.kaggle.com/c/fake-news/data
df = pd.read_csv('train.csv')

In [61]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [62]:
#removing NA values
df = df.dropna()

In [63]:
## Get the independent features
x=df.drop('label', axis =1)

In [64]:
y=df['label']

In [65]:
x.shape

(18285, 4)

In [66]:
y.shape

(18285,)

In [67]:
import tensorflow as tf

In [68]:
tf.__version__

'2.3.0'

In [69]:
## vocabulary size
vacb_size = 5000

### One hot representation

In [70]:
from tensorflow.keras.preprocessing.text import one_hot

In [71]:
messages = x.copy()

In [72]:
messages.reset_index(inplace=True) #resetting index as we removed the NA values

In [73]:
import nltk
import re
from nltk.corpus import stopwords

In [74]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vishal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data preprocessing

I am taking only title from our data set to train the model

In [75]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    text = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    text = text.lower()
    text = text.split()
    
    text = [ps.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [76]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [77]:
#onehot reprerepresentation creates the list of arrays using the vocab size we have selected

onehot_repr=[one_hot(words,vacb_size)for words in corpus]
onehot_repr

[[4763, 2882, 4552, 454, 1636, 3268, 4612, 4947, 1111, 1879],
 [1868, 2800, 1268, 4096, 3086, 2571, 4462],
 [1692, 4452, 880, 2536],
 [3274, 2309, 1262, 2776, 254, 4996],
 [1202, 3086, 2391, 4443, 981, 2782, 3086, 4832, 2049, 3768],
 [2632,
  3729,
  2824,
  2179,
  3699,
  4697,
  3522,
  1234,
  3927,
  3874,
  2726,
  2297,
  4419,
  1374,
  4462],
 [428, 3612, 1558, 1802, 582, 4126, 269, 1594, 1332, 440, 2329],
 [179, 231, 2024, 152, 4817, 2764, 4697, 1498, 1332, 440, 2329],
 [2649, 167, 1874, 4897, 2828, 2580, 2974, 4550, 4697, 1615],
 [3860, 2985, 853, 2712, 2594, 1099, 4758, 3877],
 [810, 3439, 1063, 4498, 261, 1715, 3716, 4881, 2568, 556, 23],
 [2776, 641, 1636, 2580, 4697, 4817],
 [1795, 482, 4445, 3967, 2949, 1309, 1980, 4741, 4374],
 [2743, 2630, 4055, 4531, 2906, 1047, 1677, 1332, 440, 2329],
 [4336, 4794, 432, 48, 3462, 1332, 440, 2329],
 [621, 4835, 1560, 969, 3503, 2592, 2128, 4344, 1666, 981],
 [4067, 3077, 2800],
 [4781, 2150, 1897, 3002, 4697, 2667, 882, 4462],
 [1112

In [78]:
#we have to make sure that all senteses have the same size, thats why pad sequences are in use
#I have used sequential model
#Using LSTM layer which good for the output i want
#using dense layes because i want one output at the end

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [79]:
#checked the max length of the sentense and apply the pad sequencesing
#this is the input data that our model will be trained on

sent_length =20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4947 1111 1879]
 [   0    0    0 ... 3086 2571 4462]
 [   0    0    0 ... 4452  880 2536]
 ...
 [   0    0    0 ... 1332  440 2329]
 [   0    0    0 ... 1061 4212 3011]
 [   0    0    0 ...   84 4228  653]]


In [80]:
##creating model
embedding_vector_features=40

model=Sequential()

model.add(Embedding(vacb_size,embedding_vector_features, input_length=sent_length))
#adding 100 neurons in the LSTM layer
model.add(LSTM(100))
#using sigmoid function which good for the single final o/p 
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [81]:
#creating array

import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [82]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [83]:
#spliting train and test data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [84]:
## model training
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 42ms/step - loss: 0.3324 - accuracy: 0.8422 - val_loss: 0.1960 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1404 - accuracy: 0.9438 - val_loss: 0.1990 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0988 - accuracy: 0.9621 - val_loss: 0.2236 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0723 - accuracy: 0.9762 - val_loss: 0.2786 - val_accuracy: 0.9215
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0466 - accuracy: 0.9849 - val_loss: 0.2796 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0311 - accuracy: 0.9905 - val_loss: 0.3946 - val_accuracy: 0.8968
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0197 - accuracy: 0.9947 - val_loss: 0.4060 - val_accuracy: 0.9140

## performance Metrics and Accuracy

In [85]:
y_pred=model.predict_classes(x_test)

In [86]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [87]:
confusion_matrix(y_test,y_pred)

array([[3100,  319],
       [ 189, 2427]], dtype=int64)

In [88]:
accuracy_score(y_test,y_pred)

0.915824357912179